In [ ]:
import pandas as pd
import numpy as np
from skimage import io, img_as_ubyte
from skimage.transform import rotate
import cv2
import os
import time

def rotate_and_save_images(image_dir, rotation_angle, image_list):
    """
    Rotates images by a specified angle and saves them to the same directory.
    
    Parameters:
    image_dir (str): Directory containing the images to rotate.
    rotation_angle (int): Degrees to rotate the image (1 to 360).
    image_list (list): List of image names (without extensions) to process.
    
    Returns:
    None
    """
    for image_name in image_list:
        image_path = os.path.join(image_dir, f"{image_name}.jpeg")
        
        try:
            # Load the image
            image = io.imread(image_path)
            
            # Rotate the image by the specified angle
            rotated_image = rotate(image, rotation_angle)
            
            # Save the rotated image
            rotated_image_path = os.path.join(image_dir, f"{image_name}_{rotation_angle}.jpeg")
            io.imsave(rotated_image_path, img_as_ubyte(rotated_image))
            
        except FileNotFoundError:
            print(f"Warning: File not found - {image_path}. Skipping...")
        except Exception as error:
            print(f"Error processing {image_name}: {error}")

def mirror_images(image_dir, axis, image_list):
    """
    Mirrors images along the specified axis and saves them to the same directory.
    
    Parameters:
    image_dir (str): Directory containing the images to mirror.
    axis (int): Axis to mirror along (0 for vertical, 1 for horizontal).
    image_list (list): List of image names (without extensions) to process.
    
    Returns:
    None
    """
    for image_name in image_list:
        image_path = os.path.join(image_dir, f"{image_name}.jpeg")
        
        try:
            # Load the image
            image = io.imread(image_path)
            
            # Mirror the image along the specified axis
            mirrored_image = cv2.flip(image, axis)
            
            # Save the mirrored image
            mirrored_image_path = os.path.join(image_dir, f"{image_name}_mirrored_{axis}.jpeg")
            io.imsave(mirrored_image_path, img_as_ubyte(mirrored_image))
            
        except FileNotFoundError:
            print(f"Warning: File not found - {image_path}. Skipping...")
        except Exception as error:
            print(f"Error processing {image_name}: {error}")

if __name__ == '__main__':
    # Measure script execution time
    start_time = time.time()

    # Load the dataset
    labels_file_path = r"C:\Users\janan\OneDrive\Desktop\AI\AI Project\Final Project\diabetic-retinopathy-detection\trainLabels_original.csv"
    train_labels = pd.read_csv(labels_file_path)
    
    # Remove the '.jpeg' extension from image names
    train_labels['image'] = train_labels['image'].str.rstrip('.jpeg')
    
    # Separate images into two groups based on their DR levels
    images_no_dr = train_labels[train_labels['level'] == 0]['image'].tolist()
    images_with_dr = train_labels[train_labels['level'] >= 1]['image'].tolist()

    # Directory containing the resized images
    resized_image_dir = './train_resized/resized-256/'

    # Mirror images with no DR
    print("Mirroring Non-DR Images...")
    mirror_images(resized_image_dir, axis=1, image_list=images_no_dr)

    # Rotate images with DR by different angles
    print("Rotating DR Images...")
    for angle in [90, 120, 180, 270]:
        print(f"Rotating by {angle} degrees...")
        rotate_and_save_images(resized_image_dir, rotation_angle=angle, image_list=images_with_dr)
    
    # Mirror images with DR
    print("Mirroring DR Images...")
    mirror_images(resized_image_dir, axis=0, image_list=images_with_dr)

    # Print completion message and execution time
    print("Image augmentation completed!")
    print(f"--- {time.time() - start_time:.2f} seconds ---")
